## Introduction

The purpose of our software and of automatic differentiation is to efficiently evaluate derivatives at machine precision.
There are two main alternatives to automatic differentiation ("AD") for the evaluation of derivatives: symbolic differentiation and numerical (approximation) methods.
While symbolic differentiation evaluates derivatives to machine precision, it is often costly to evaluate and prone to 'expression swell': 
symbolic differentiation may result in exponentially large expressions to evaluate. In contrast, numerical methods (such as the finite difference method) are 
quick to evaluate but do not generally result in machine precision: when evaluating $(f(x+\epsilon)-f(x))/\epsilon$, use of a large epsilon results in imprecise approximation,
whereas use of a small epsilon results in floating point errors. Moreover, the complexity of numerical methods increases as the number of dimensions of the gradient increases.$^{3}$
Automatic differentiation circumvents these challenges by "accumulating" numerical evaluations of simple elementary functions.
    
The efficient evaluation of derivatives at machine precision naturally has a wide (and growing) range of application areas, of which a few are highlighted below:
* Differentiation of functions implemented as code: AD does not require closed-form expressions (as is the case in symbolic differentiation)$^{3}$
* Backpropagation: training neural networks requires finding weights that minimize the objective function
    * AD has seen increasingly widespread use since ~2015 as deep learning grows in popularity (e.g. AD implementations in TensorFlow and PyTorch)$^{3}$
    * Applications include computer vision, NLP
* Solving ODEs
* Gradient-based optimization (finding minima through step-wise updates)$^{3}$
* Root finding, including Newton's method

## Background

Suppose we have a function $f: \mathbb{R}^n \to \mathbb{R}^m$
$$
f(\mathbf{x})=
\begin{bmatrix}
    f_1(x_1, ..., x_n) \\
    f_2(x_1, ..., x_n) \\
    \vdots \\
    f_m(x_1, ..., x_n)
\end{bmatrix}
$$
and suppose that we seek ${\frac{\partial f}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$.

If we have $J_f$, the Jacobian matrix of $f$, we can compute ${\frac{\partial f}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$ as the product of the Jacobian matrix and the desired seed vector $s$:
$$
J_f\mathbf{s}=
\begin{bmatrix}
    \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \dots & \frac{\partial f_1}{\partial x_n} \\[1ex]
    \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \dots & \frac{\partial f_2}{\partial x_n} \\[1ex]
    \vdots & \vdots & \ddots & \vdots \\[1ex]
    \frac{\partial f_m}{\partial x_1} & \frac{\partial f_m}{\partial x_2} & \dots & \frac{\partial f_m}{\partial x_n}
\end{bmatrix}
\begin{bmatrix}
1 \\[1ex]
0 \\[1ex]
\vdots \\[1ex]
0
\end{bmatrix}
=
\begin{bmatrix}
\frac{\partial f_1}{\partial x_1} \\[1ex]
\frac{\partial f_2}{\partial x_1} \\[1ex]
\vdots \\[1ex]
\frac{\partial f_m}{\partial x_1}
\end{bmatrix}
$$

Rather than using matrix-vector multiplication or an explicit matrix representation of the Jacobian, the forward mode of AD involves calculating ${\frac{\partial f}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$ 
(i.e.  a single column of the Jacobian) by computing ${\frac{\partial f_i}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$ for each of $i=1,...,m$ separately in one or more *forward "passes"*
(also sometimes referred to as "sweeps").$^{2, 3}$ If $f: \mathbb{R}^1 \to \mathbb{R}^m$, only one forward pass would be needed to calculate the full Jacobian, since in that case $n=1$, the Jacobian matrix is $m$x$1$,
and the seed vector would have a single entry. If $f: \mathbb{R}^n \to \mathbb{R}^m$, then $n$ forward passes would be required (one forward pass for each entry in the $n$x$1$ seed vector). 
Although the matrix representation of the Jacobian is not explicitly used in the forward mode, this intuition carries over: within each forward pass, the derivative of
one of the variables is set to 1 and the rest to zero (for example, $\dot{x}_1=1$ if we seek ${\frac{\partial f}{\partial x_1}}$)

AD works by breaking down the function $f$ into a composition of much simpler elemental functions (or elemental operations).
Each forward pass starts by assigning 1 to the derivative of the variable $x_i$ for which we want to differentiate $f$ with respect to, and assigning 0 to all other variables.
Then the elemental functions are evaluated from innermost to outermost, accumulating two numerical values at each intermediate variable $v_i$ in the trace$^{3, 4}$:
* the **"primal"**: the value of the elemental function at $\mathbf{x}=\mathbf{a}$, denoted $v_i$ (i.e. the input of the elemental function will be carried over from any inner functions evaluated earlier in the trace)
* the **derivative** or "tangent", denoted $\dot{v_i}$ (i.e. the derivative of the elemental function evaluated at $\mathbf{x}=\mathbf{a}$)

Within each step, the derivative $\dot{v_i}$ is implicitly calculated using the chain rule. For example, if the elemental function corresponding to $v_3$ results from the composition of several functions $w_3(w_2(\mathbf{x}), w_1(\mathbf{x}))$ which for clarity we denote $w_3(u_1, u_2)$ then:
$$
\dot{v_3} = {\frac{\partial}{\partial x_1}} w_3(w_2(\mathbf{x}), w_1(\mathbf{x})) = {\frac{\partial w_3}{\partial u_1}}{\frac{\partial w_2}{\partial x_1}} + {\frac{\partial w_3}{\partial u_2}}{\frac{\partial w_1}{\partial x_1}}
$$
In forward mode, the innermost elemental function primals and derivatives are evaluated first and 'accumulated' outwards.

## Software Organization
Below is a diagram of our directory stucture. 

The main project directory contains the `build.sh` file. This file has the `cmake` and `make install` commands that build the test suite in `AutoDiff/tests/`. The `build.sh` file is specified in `.travis.yml`, enabling TravisCI to build our tests each time we `git push`. Our test suite also be utilized by `CodeCov` to show how much of our code we've hit.

We have two main ways to package our software: `cmake` and `make`. `CMake` is a lot nicer because the user has to do so much less in order to link up our library. `Make` requires us to configure the `VPATH` and locate the `.so` file. `CMake` requires less user configuration. 

Our `AutoDiff/` folder is what the user needs to have locally in order utilize our AutoDiff library. The user will need to `cd` into the `build/` folder, run the `cmake` command, then edit the `Makefile`, adding the `target_link_libraries` command to link up our library. Then, all the user needs to do is add `#include <AutoDiff.h>` at the top of their `.cpp` file, and `C++` will know where to find our AutoDiff library.

We have two modules: one for differentiation and one for root-finding, where root-finding actually uses the implementation of the differentiation module. A third module, a main module, will tie the two modules together.
```
cs107-FinalProject/
            | README.md
            | build.sh
            | .travis.yml
            | AutoDiff/
                    | README.md
                    | docs/
                        | README.md
                        | main_doc.md
                        | Doxygen.config
                        | images/
                    | include/
                        | AutoDiff.h
                    | src/
                        | AutoDiff_main.cpp
                        | diff.cpp
                        | roots.cpp
                    | tests/
                        | CMakeList.txt
                        | test_main.cpp
                        | src/
                            | test1.cpp
                            | test2.cpp
                            | test3.cpp
                    | lib/
                    | build/
```

## How to Use
We assume that our user has followed procedures for downloading the `AutoDiff/` folder, run `cmake` within the `build/` folder, then linked the library by editing the `Makefile` to include `target_link_libraries`.

Now that we've made and linked our libraries, `g++` compiler knows where to find the `AutoDiff` library. All they need to do is `include` the autodiff header file `AutoDiff.h` at the top of their `cpp` file where they intend to call/utilize the `AutoDiff` library.

Next, the user has to declare some `autodiff types` (described in the implementation section). They construct a composite function of `autodiff types` and `.` call the differentiation and root-finding methods on that function, assigning it to some local variable for storagign the computation.

## Implementation
The core of our diff implementation lives in `diff.cpp`, and it requires type declarations and function definitions. Specifically, defining new `autodiff types` for the operators/functions `add`, `subtract`, `multiply`, `divides`, `sin`, `cos`, `tan`, `arcsin`, `arccos`, `arctan`, `power`, etc. We can define types along with function declarations in `AutoDiff.h`, then reference the types and define the functions in `diff.cpp`. The functions we implement that take in `autodiff types` and numbers (`floats`) and output two values: the function evaluation and the derivative evaluation.

At a high level, the user creates some function that is a composition of `autodiff types`. Since the user is expected to be working with `autodiff types`, we have to do no string parsing that would otherwise be necessary.

The `autodiff type` variables will be seeded within `diff.cpp`. Since vectors of `autodiff type` are also possible, our functions are initialized to return arrays of numbers as the function evaluation and derivative evaluation. A single function is still a specific case of the Jacobian, so we will presume that we are always outputting some matrix. For instance the function `F = (f, g)` should output (in pseudocode) `[[f.val | x1 and x2, g.val | x1 and x2], [[F.dfdx1, F.dfdx2], [F.dgdx1, F.dgdx2]]`, giving first the function evaluation as the first element and the derivative evaluation as the second element.

The function implementations "sort of recursively" call one another so that the inner most level is returned, allowing the next nested layer to get the `.val` and the `.dv`, up and up (like Russian dolls), until the outer most function is eventually computed. By this implementation, we essentially get the forward mode table that we did in class.

Let's take an example of how the functions work. Suppose we have the function `sin(cos(x**2))` where `x.val` is seeded as `1.0` and `x.dv` is seeded as `1.0`. What forward mode does is expand the function derivative from inside out. So, first the `power` function implementation will return the function value and derivative value `[x.val**2 = 1**2 = 1, 2*x.val*x.dv = 2*1*1 = 2]` Next, the `cos` function returns `[cos((x**2).val) = cos(1), -sin(x.val**2)*(x**2) = -sin((x**2).val)*((x**2).dv) = -sin(1)*2 = -2sin(1)]`. Next the `sin` function returns `[sin(cos(x**2).val) = sin(cos(1)), cos(cos(x**2).val)*cos(x**2).dv = cos(cos(1))*(-2sin(1))]`.

Once our AD is implemented within `diff.cpp` we essentially just need to reference the functionality to write our root finder. The Newton-Raphson method is pretty short, and involves interating once we acquire the function evaluation and derivative evaluation. The file `roots.cpp` will handle this iteration.

Lastly, we will wrap the diff implementation along with the root finder in a file called `AutoDiff_main.cpp`.

## Testing

* Location：under a folder name "tests" in the GitHub project repo

* Tools:

    * CodeCov: 
    
        CodeCov provides statistics of code coverage so that each commit and pull request is measured and in control. 
        To do this, we need to create a .yaml file and specify our metrics (target at least 90%). 
        As our software covers two parts of functions: the basic function(AD implementation) and extensions, 
        it is reasonable to use tags in the .yaml file to separate coverage reports for different modules$^7$. 
        Setting Codecov to send notifications to the Slack channel is also a good idea$^8$, so everyone in the group 
        gets notified when there is an update.
    
    * TravisCI: 
        
        TravisCI is a continuous integration platform that automatically builds and tests code changes. 
        Instead of merging large pieces of code and integrate them all in the end, we want to take a continuous 
        integration approach - merge in small code changes and get immediate feedback when each commit and pull 
        request happens$^9$. We also need a .yaml file to use it. It can also send notifications through Slack. 
* Plan: 
    * Step 1: Exploratory Testing

        Exploratory testing is a simultaneous process of test design and test execution all done at the same time, 
        contrarily to scripted testing, where all test cases are determined in advance$^{10}$.
        We will note down ideas about what to test and how it can be tested in a document file (.md). 
        We will update this document with additional test case designs, critically evaluate current test results with 
        respect to our expectations, and learn from the testing constantly.

    * Step 2: Unit tests

        Unit tests isolate issues that may arise and each unit is tested independently. 
        We will define the smallest unit to a each Class method. 
        By writing tests for the smallest testable units, then the compound behaviors between those, 
        we can build up comprehensive tests for more complex applications.$^{11}$

    * Step 3: Acceptance tests

        After unit testings, we want to validate software against functional requirements/specifications. We mainly concentrate on:

        * Functionality

        * Basic Usability: Can users freely navigate through the screens without any difficulties?

        * Error Conditions: Do suitable error messages display? Are they clear and proper?

        * Accessibility


        

* Evaluations:

    * Code coverage

    * Risk analysis: What are the potential risks of current implementation? What are the important ones and what need to be addressed? 

    * Test execution log: recordings on the test execution

## Documentations

* Developer diary:

    * Developer diary is an invaluable tool that helps the team remember bits of information that will otherwise get washed away. There is no need to keep it up to date. It can be records of: resources (url), insights (from success/failures), ideas, solutions, questions, and minutes.

* Commit early, commit often

* Software documentations: doxygen and graphviz packages

* Demo examples: In the end, a few demo examples in the GitHub repo will let users get started easily. 

## Extensions

### Dual Numbers

Dual number is an interesting type of number that uses the symbol $\epsilon$ (epsilon) and offers how to do automatic
differentiation in another persepective. It automatically calculates the derivative and the value of a function at the same time! 

Dual numbers are pretty similar to imaginary numbers. A dual number has a real part and a dual part. We write

$$
x = a + b\epsilon
$$

But! For imaginary numbers, $i^2 = -1$, whereas for dual numbers, $\epsilon^2 = 0$ (and $\epsilon$ is not 0!). 

#### Basic Math

Add and subtraction of dual numbers are the same as adding complex numbers: we just add the real and dual parts separately.

$$
(3+6\epsilon) + (1+2\epsilon) = (3+1)+(6\epsilon+2\epsilon) = 4+8\epsilon
$$

$$
(3+6\epsilon) - (1+2\epsilon) = (3-1)+(6\epsilon-2\epsilon) = 2+4\epsilon
$$

To multiply dual numbers, we use F.O.I.L. just like we do with complex numbers

$$
(3+6\epsilon) \times (1+2\epsilon) = 3 + 6\epsilon +6\epsilon + 12\epsilon^2 = 3 + 12\epsilon + 12\epsilon^2
$$

As $\epsilon^2=0$, the last item $12\epsilon^2$ disappears. 

$$
(3+6\epsilon) \times (1+2\epsilon) = 3 + 12\epsilon
$$

The division process relates to the conjugate, like complex division - multiplying the denominator  by its conjugate 
in order to cancel the non-real parts.$^{12}$ The formula is given below:

$$
\begin{aligned}
\frac{a+b\epsilon}{c+d\epsilon} &= \frac{(a+b\epsilon)(c-d\epsilon)}{(c+d\epsilon)(c-d\epsilon)}\\
&= \frac{ac-ad\epsilon+bc\epsilon-bd\epsilon^2}{c^2 - d^2\epsilon^2}\\
&= \frac{ac-ad\epsilon+bc\epsilon-bd\epsilon^2}{c^2}\\
&= \frac{ac+(bc-ad)\epsilon}{c^2}\\
&= \frac{a}{c}+\frac{(bc-ad)\epsilon}{c^2}
\end{aligned}
$$

which is defined when $c$ is non-zero. If, on the other hand, $c$ is zero while $d$ is not, then the equation

$$
\begin{aligned}
\frac{a+b\epsilon}{c+d\epsilon} &= k\\
\frac{a+b\epsilon}{d\epsilon} &= k\\
a+b\epsilon &= kd\epsilon\\
\end{aligned}
$$

1. has no solution if $a$ is nonzero
2. otherwise is solved by any dual number of the form $k=\frac{b}{d}+y\epsilon$. To expalins this, suppose we have $k=x+y\epsilon$: 

$$
\begin{aligned}
b\epsilon &= kd\epsilon\\
b\epsilon &= (x+\epsilon y)d\epsilon\\
b\epsilon &= xd\epsilon + \epsilon^2 y\\
b\epsilon &= xd\epsilon\\
x &= \frac{b}{d}\epsilon
\end{aligned}
$$

So, $k=\frac{b}{d}+y\epsilon$ and is the value of $\frac{a+b\epsilon}{c+d\epsilon}$.

#### Using $\epsilon^2$ for AD

Suppose we have a function: $
f(x) = 3x + 2$, and calculates $f(4)$ and $f'(4)$. 

We know $4 = 4 + 1\epsilon$, using $1$ for the dual component, since $x$ has a derivative of $1$.$^{13}$ We take this into the function:

$$
f(4) = f(4 + \epsilon) = (4 + 1\epsilon) \times 3 + 2 + 1\epsilon = (12 + 3\epsilon) + 2 = 14 + 3\epsilon
$$

We can find out the real number component $(14)$ is the value of $f(4)$ and the dual component $(3)$ is the derivative $f'(4)$, 
which is verified below:

$$
f(x) = 3x+2 = 3\times4 + 2 = 14
$$

$$
f'(x) = 3
$$

It is not an coincidence! Now try a quadratic example: $f(x)=5x^2 + 3x + 1$, also evaluates at $f(4)$.

$$
\begin{aligned}
f(4 + \epsilon) &=5(4 + \epsilon)^2 + 3\times(4 + \epsilon) + 1 \\
 &= 5(16 + 8\epsilon+\epsilon^2 ) + 12 + 3\epsilon + 1\\
 &= 80 + 40\epsilon + 5\epsilon^2 + 12 + 3\epsilon + 1\\
 &= (80+12+1) + (40+3)\epsilon + 5\epsilon^2 \\
 &= 93 + 43\epsilon + 5\epsilon^2\\
 &= 93 + 43\epsilon
\end{aligned}
$$

Now verify it:

$$
\begin{aligned}
f(4)&=5\times 4^2 + 3\times 4 +1\\ &= 5\times 16 + 12 + 1\\ &= 80 + 12 + 1 \\&= 93
\end{aligned}
$$

$$
\begin{aligned}
f'(x)&=10x + 3\\ 
f'(4)&=10\times 4 + 3 = 43
\end{aligned}
$$

We can also use dual number on trigonometric functions to do differentiation. 
What's more, it is not only limited to the 1st derivative.
It is possible to modify this technique to get the 2nd the 3rd, or the Nth derivative of a function. But in a 2nd derivative
scenario, we need to define $\epsilon^3=0$, instead of $\epsilon^2=0$, so on and so forth.
We can also extend this to multivariable functions. To do this, we need to define $\epsilon_x$, $\epsilon_y$, ... for each variables.

To conclude, using dual numbers gives us exact derivatives, within the limitations of floating point math 
(compared to finite difference method). 

#### Implementation

It is tempting to create a "DualNumber" Class and overloads different operators$(+,-,*,/)$ and math functions, eg. sqrt, pow, sin, cos, tan, atan, ... 
We keep all independent variables in a array-like container and in each overloading function, we loop through this array to calcualte derivative for each variables.

### Root Finding and Nonlinear Sets of Equations

There are many useful applications of automatic differentiation and we would love to code them. As we get started, we do not want to be too ambitious so for this part, we only list out ideas. 
More detailed plans and implementation may be presented in the next milestone.

* Newton-Raphson Method using derivative: rapid local convergence but unpredictable global 
convergence. We can visualize the convergence graph and see fractals, which refers 
to self-similar shapes across different scales

* Halley’s Method (extension of Newton’s method)

* Globally Convergent Methods for Nonlinear Systems of Equations

### Solution of Linear Algebraic Equations

Current popular methods of solving the problems below mostly take an iterative approach, 
e.g. Gauss-Jordan Elimination, Gaussian Elimination with Backsubstitution, LU Decomposition. Solving these problems also take large computation time; we are curious if automatic differentiation will give a better solution.

* Solution of the matrix equation $Ax=b$ for an unknown vector $x$
* Solution of more than one matrix equation $Ax_j=b_j$, for a set of vectors $x_j$, $j=0,1,..,$ each corresponding to 
a different, known right-hand side vector bj
* Calculation of the matrix $A^{-1}$ that is the matrix inverse of a square matrix $A$, i.e., $A^{-1}\cdot A=A \cdot A^{-1} =I$, 
where $I$ is the identity matrix   
* Singular value decomposition of a matrix $A$
* Linear least-squares problem

    The reduced set of equations to be solved can be written as the $N\times N$ set of equations
    
    $$
    (A^{T}\cdot A)\cdot x = (A^{T}\cdot b)
    $$

    where $A^{T}$ denotes the transpose of the matrix A.
    These quations are called the normal equations of the linear least-squares problem. 

Usually, a direct solution of the normal equations is **NOT** generally the best way to find least-squares solutions.
We would like to see how automatic differentiation performs in this task and compare the differences. 

## References

1. http://web4.cs.ucl.ac.uk/staff/D.Barber/publications/AMLAutoDiff.pdf
2. http://www.robots.ox.ac.uk/~tvg/publications/talks/autodiff.pdf
3. "Automatic Differentiation in Machine Learning: A Survey" https://www.jmlr.org/papers/volume18/17-468/17-468.pdf
4. https://harvard-iacs.github.io/2020-CS107/lectures/lecture10/notebook/
5. https://kenndanielso.github.io/mlrefined/
6. https://people.cs.umass.edu/~domke/courses/sml/09autodiff_nnets.pdf
7. https://docs.codecov.io/docs/flags
8. https://docs.codecov.io/docs/notifications 
9. https://docs.travis-ci.com/user/for-beginners/
10. https://www.guru99.com/exploratory-testing.html
11. https://en.wikipedia.org/wiki/Unit_testing#:~:text=Unit%20tests%20are%20typically%20automated,an%20individual%20function%20or%20procedure
12. https://en.wikipedia.org/wiki/Dual_number#:~:text=Division%20of%20dual%20numbers%20is,cancel%20the%20non%2Dreal%20parts
13. https://blog.demofox.org/2017/02/20/multivariable-dual-numbers-automatic-differentiation/